In [1]:
# =================== IMPORTS ===================
import os
import math
import copy
import torch
import torch.nn as nn
import torch.optim as optim
import transformers
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from torch.utils.data import Dataset, DataLoader

# =================== SETUP ===================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(64)

save_path = 'Training_results/Combined_5foldcv_BERT_Multihead_512/'
os.makedirs(save_path, exist_ok=True)

tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = transformers.BertModel.from_pretrained('bert-base-uncased')

# =================== UTILS ===================
def haversine(lat1, lon1, lat2, lon2):
    R = 3959
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def get_box_area(lat1, lon1, lat2, lon2):
    side1 = haversine(lat1, lon1, lat1, lon2)
    side2 = haversine(lat1, lon1, lat2, lon1)
    return side1 * side2

def get_target_list():
    return [
        'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage',
        'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible',
        'replacementAssistanceEligible', 'personalPropertyEligible'
    ]

# =================== DATA HANDLING ===================
def clean_tweets(df, storm_name, size_threshold=80):
    df = df[df['storm_name'] == storm_name] if 'storm_name' in df.columns else df
    bboxes = df.place_bbox.apply(lambda x: [[float(i.strip('()[]')) for i in x.split(', ')][i] for i in [1, 0, 3, 2]])
    areas = bboxes.apply(lambda x: get_box_area(*x))
    df = df.loc[((df.geo.apply(lambda x: 'Point' in str(x))) | (areas < size_threshold)), :]
    df['zip_code'] = pd.to_numeric(df['zip_code'], errors='coerce')
    df = df[df['zip_code'].notna()]
    df['zip_code'] = df['zip_code'].astype(int)
    return df.groupby('zip_code')

def load_all_storm_data():
    target_list = get_target_list()

    # Load and group Harvey tweets and targets
    tweets_harvey = pd.read_csv('harvey_corrected.csv')
    tweets_harvey.rename(columns={'zipcode': 'zip_code'}, inplace=True)
    tweet_grouped_harvey = clean_tweets(tweets_harvey, 'harvey')
    targets_harvey = pd.read_csv('disaster_4332.csv')
    target_grouped_harvey = targets_harvey.groupby('damagedZipCode')

    # Load shared tweet file for Imelda, Beryl, Laura, Hanna
    tweets_all = pd.read_csv('organized_with_zipcode.csv')

    tweet_grouped_imelda = clean_tweets(tweets_all, 'imelda')
    targets_imelda = pd.read_csv('disaster_4466.csv')
    target_grouped_imelda = targets_imelda.groupby('damagedZipCode')

    tweet_grouped_beryl = clean_tweets(tweets_all, 'beryl')
    targets_beryl = pd.read_csv('disaster_4798.csv')
    target_grouped_beryl = targets_beryl.groupby('damagedZipCode')

    tweet_grouped_laura = clean_tweets(tweets_all, 'laura')
    tweet_grouped_hanna = clean_tweets(tweets_all, 'hanna')

    # Organize tweet + target group pairs
    all_tweet_groups = [
        (tweet_grouped_harvey, target_grouped_harvey),
        (tweet_grouped_imelda, target_grouped_imelda),
        (tweet_grouped_beryl, target_grouped_beryl),
        (tweet_grouped_laura, None),  # No ground truth target file
        (tweet_grouped_hanna, None)   # No ground truth target file
    ]

    tweet_dict, target_dict = {}, {}

    for tweet_grouped, target_grouped in all_tweet_groups:
        for zip_code, group in tweet_grouped:
            tweet_dict[int(zip_code)] = group['text']

        if target_grouped is not None:
            for zip_code, group in target_grouped:
                target_dict[int(zip_code)] = group[target_list]
        else:
            for zip_code in tweet_grouped.groups.keys():
                target_dict[int(zip_code)] = pd.DataFrame([[0] * len(target_list)], columns=target_list)

    return tweet_dict, target_dict


def prepare_data(tweet_dict, target_dict):
    intersect = list(set(tweet_dict.keys()) & set(target_dict.keys()))
    texts = ['\n'.join(tweet_dict[z].tolist()) for z in intersect]
    labels = [target_dict[z].sum().apply(lambda x: 1 if x > 0 else 0).values for z in intersect]
    return texts, labels

# =================== MODEL DEFINITIONS ===================
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        encoding = self.tokenizer.encode_plus(
            self.texts[idx],
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float32)
        }

class BERTDeepMultiHeadClassifier(nn.Module):
    def __init__(self, num_targets=11, hidden_dim=256):
        super().__init__()
        self.bert = model_bert
        self.drop = nn.Dropout(0.3)
        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.bert.config.hidden_size, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, 1)
            ) for _ in range(num_targets)
        ])

    def forward(self, input_ids, attention_mask):
        pooled = self.bert(input_ids=input_ids, attention_mask=attention_mask).pooler_output
        x = self.drop(pooled)
        return torch.cat([torch.sigmoid(head(x)) for head in self.heads], dim=1)

# =================== TRAINING LOOP ===================
def run_kfold_cv(texts, labels, target_list, max_len=512, batch_size=24, num_epochs=100):
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    saved_models = []
    fold_scores = []

    for fold, (train_idx, val_idx) in enumerate(kfold.split(texts, labels)):
        print(f"\n--- Fold {fold + 1} ---")
        X_train, X_val = [texts[i] for i in train_idx], [texts[i] for i in val_idx]
        y_train, y_val = [labels[i] for i in train_idx], [labels[i] for i in val_idx]

        train_loader = DataLoader(CustomDataset(X_train, y_train, tokenizer, max_len), batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(CustomDataset(X_val, y_val, tokenizer, max_len), batch_size=batch_size, shuffle=False)

        model = BERTDeepMultiHeadClassifier(num_targets=len(target_list)).to(device)
        optimizer = optim.Adam(model.parameters(), lr=1e-5)
        criterion = [nn.BCELoss() for _ in range(len(target_list))]

        best_f1, best_model_path = 0, os.path.join(save_path, f"fold_{fold+1}.pth")

        for epoch in range(num_epochs):
            model.train()
            for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} Training"):
                optimizer.zero_grad()
                inputs = batch['input_ids'].to(device)
                masks = batch['attention_mask'].to(device)
                labels_ = batch['labels'].to(device)
                outputs = model(inputs, masks)
                loss = sum(criterion[i](outputs[:, i], labels_[:, i]) for i in range(len(target_list)))
                loss.backward()
                optimizer.step()

            # Validation
            model.eval()
            preds, true = [], []
            with torch.no_grad():
                for batch in tqdm(val_loader, desc=f"Epoch {epoch+1} Validation"):
                    inputs = batch['input_ids'].to(device)
                    masks = batch['attention_mask'].to(device)
                    labels_ = batch['labels'].to(device)
                    outputs = model(inputs, masks)
                    preds.append(outputs.cpu().numpy())
                    true.append(labels_.cpu().numpy())

            preds = np.vstack(preds)
            true = np.vstack(true)
            f1 = np.mean([f1_score(true[:, i], preds[:, i] > 0.5) for i in range(len(target_list))])
            print(f"Epoch {epoch+1} F1: {f1:.4f}")

            if f1 > best_f1:
                best_f1 = f1
                torch.save(model.state_dict(), best_model_path)
                print(f"New best model saved: Fold {fold+1}, F1={best_f1:.4f}")

        fold_scores.append(best_f1)
        saved_models.append(best_model_path)

    return fold_scores, saved_models

# =================== MAIN ===================
if __name__ == "__main__":
    tweet_dict, target_dict = load_all_storm_data()
    texts, labels = prepare_data(tweet_dict, target_dict)
    target_list = get_target_list()
    results, models = run_kfold_cv(texts, labels, target_list)
    print("\nCross-validation complete.")
    print("F1 scores across folds:", results)


/tmp/job.15459737/ipykernel_1244491/675523239.py:65: DtypeWarning: Columns (9,10,13,28,46,49) have mixed types. Specify dtype option on import or set low_memory=False.
  targets_harvey = pd.read_csv('disaster_4332.csv')
/tmp/job.15459737/ipykernel_1244491/675523239.py:72: DtypeWarning: Columns (28,46,49) have mixed types. Specify dtype option on import or set low_memory=False.
  targets_imelda = pd.read_csv('disaster_4466.csv')
/tmp/job.15459737/ipykernel_1244491/675523239.py:76: DtypeWarning: Columns (9,13,28,46,49) have mixed types. Specify dtype option on import or set low_memory=False.
  targets_beryl = pd.read_csv('disaster_4798.csv')



--- Fold 1 ---


Epoch 1 Validation: 100%|██████████| 8/8 [00:04<00:00,  1.79it/s]


Epoch 1 F1: 0.0000


Epoch 2 Validation: 100%|██████████| 8/8 [00:04<00:00,  1.81it/s]


...

Epoch 98 F1: 1.0000


Epoch 99 Validation: 100%|██████████| 8/8 [00:03<00:00,  2.15it/s]


Epoch 99 F1: 0.9972


Epoch 100 Validation: 100%|██████████| 8/8 [00:03<00:00,  2.23it/s]

Epoch 100 F1: 0.9972

Cross-validation complete.
F1 scores across folds: [np.float64(0.3296380341903035), np.float64(0.8992496207969354), np.float64(0.9355607146736179), np.float64(0.982087542087542), np.float64(1.0)]
